#CEIA - IIA - Trabajo Practico 1 - Torres de Hanoi

##Grupo 1 - 
### Miembros:
- Martín Brocca <martinbrocca@gmail.com>
- Emiliano Iparraguirre <emiliano.iparraguirre22@gmail.com>
- Natalia Espector <nataliaespector@gmail.com>
- Agustín Lopez Fredes <agustin.lopezfredes@gmail.com>
- Fernando Martinez <fgmartinez1989@gmail.com>

## Resolucion del problema

In [2]:
# importar librerias requeridas
from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.hanoi_states import StatesHanoi
from aima_libs.hanoi_states import ProblemHanoi
import heapq
from tabulate import tabulate

### Búsqueda en anchura, vista en clase:

In [3]:
# Solucion provista por el profesor:

def breadth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Creamos una cola FIFO con el nodo inicial
    frontier = [NodeHanoi(problem.initial)]  

    # Creamos el set con estados ya visitados
    explored = set()
    
    node_explored = 0
    
    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node.expand(problem):
            # Solo si el estado del nodo no fue explorado
            if next_node.state not in explored:
                frontier.insert(0, next_node)

    # Si no se encontro la solución, devolvemos la métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics

### Implementación búsqueda en profundidad

In [4]:
# Solucion provista primero en profundidad (basica):

def depth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Creamos una cola LIFO con el nodo inicial
    frontier = [NodeHanoi(problem.initial)]  

    # Creamos el set con estados ya visitados
    explored = set()
    
    node_explored = 0
    
    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node.expand(problem):
            # Solo si el estado del nodo no fue explorado
            if next_node.state not in explored:
                frontier.append(next_node)

    # Si no se encontro la solución, devolvemos la métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics

### Implementación búsqueda en profundidad optimizado de manera que no mueva dos veces seguida la misma pieza

In [5]:
# En esta version del algoritmo miro hacia adelante el proximo paso, y me aseguro que no mueva la pieza que moví recien:
# Se aprovecha la librería hanoi_states library para analizar el movimiento.

from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.hanoi_states import StatesHanoi, ActionHanoi, ProblemHanoi

def depth_first_search_smart(number_disks=5):
    # Hanoi problem definition and initialization
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # LIFO queue with initial node inside
    frontier = [NodeHanoi(problem.initial)]  

    # Set for visited states
    explored = set()
    
    node_explored = 0
    
    while frontier:
        node = frontier.pop()
        node_explored += 1
        
        # Add the state to the explored set to avoid duplicates
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Have we reached the goal?
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Expand the node and filter successors
        for next_node in node.expand(problem):
            if next_node.state not in explored:
                # Avoid moving the same disk as in the previous step
                if not is_same_disk_moved(node, next_node):
                    frontier.append(next_node)

    # If no solution is found, return metrics
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth if frontier else 0, 
        "cost_total": None,
    }
    return None, metrics

def is_same_disk_moved(current_node: NodeHanoi, next_node: NodeHanoi) -> bool:
    """
    Check if the same disk was moved in the current and next nodes.
    Returns True if the same disk is moved, False otherwise.
    """
    if current_node.parent is None:  # Root node has no previous move
        return False
    
    # Get the disk moved in the current node (from its action)
    last_action = current_node.action
    if last_action is None or last_action.disk is None:
        return False
    last_moved_disk = last_action.disk
    
    # Get the disk moved in the next node (from its action)
    next_action = next_node.action
    if next_action is None or next_action.disk is None:
        return False
    next_moved_disk = next_action.disk
    
    return last_moved_disk == next_moved_disk

### Implementaión búsqueda Greedy

In [6]:
def heuristica(state, goal_state):
    # Contar cuántos discos aún no están en las torres destino (rods[1] y rods[2])
    goal_disks = set(goal_state.rods[1] + goal_state.rods[2])
    return sum(1 for rod in state.rods[:2] for disk in rod if disk not in goal_disks)

In [7]:
def busqueda_greedy(number_disks=5):
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)
    
    frontier = []
    heapq.heappush(frontier, (0, NodeHanoi(problem.initial)))  # (priority, node)
    explored = set()
    
    node_explored = 0
    
    while frontier:
        _, node = heapq.heappop(frontier)
        node_explored += 1
        
        if problem.goal_test(node.state):
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        explored.add(node.state)
        
        for next_node in node.expand(problem):
            if next_node.state not in explored:
                valor_heuristica = heuristica(next_node.state, goal_state)
                heapq.heappush(frontier, (valor_heuristica, next_node))
    
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth,
        "cost_total": None,
    }
    return None, metrics

### Implementaión búsqueda A<sup>*</sup>

In [8]:
def heuristica(state, goal_state):
    # Contar cuántos discos aún no están en las torres destino (rods[1] y rods[2])
    goal_disks = set(goal_state.rods[1] + goal_state.rods[2])
    return sum(1 for rod in state.rods[:2] for disk in rod if disk not in goal_disks)



def busqueda_a_estrella(number_disks=5):
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)
    
    frontier = []
    heapq.heappush(frontier, (0, NodeHanoi(problem.initial)))  # (priority, node)
    explored = set()
    
    node_explored = 0
    
    while frontier:
        _, node = heapq.heappop(frontier)
        node_explored += 1
        
        if problem.goal_test(node.state):
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        explored.add(node.state)
        
        for next_node in node.expand(problem):
            if next_node.state not in explored:
                costo = next_node.state.accumulated_cost
                valor_heuristica = heuristica(next_node.state, goal_state)
                f = costo + valor_heuristica
                heapq.heappush(frontier, (f, next_node))
    
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth,
        "cost_total": None,
    }
    return None, metrics


## Pruebas de ejecucion:

In [9]:
solution_anchura, metrics_anchura = breadth_first_search(number_disks=5)

In [10]:
solution_prof, metrics_prof = depth_first_search(number_disks=5)

In [11]:
solution_prof_smart, metrics_prof_smart = depth_first_search_smart(number_disks=5)

In [12]:
solution_greedy, metrics_greedy = busqueda_greedy(number_disks=5)

In [13]:
solution_a_estrella, metrics_a_estrella = busqueda_a_estrella(number_disks=5)

## Visualización de los resultados

In [14]:
#crear un arreglo de metricas para generar una sola tabla
all_metrics = [metrics_anchura, metrics_prof, metrics_prof_smart, metrics_greedy, metrics_a_estrella]

# Encabezado de la tabla:
headers = list(metrics_anchura.keys())

# Datos de la tabla - En cada fila van los valores de una metrica del diccionario
table_data = [[metrics[i] for i in headers] for metrics in all_metrics]

# Agrego las filas con los nombres de las variables (optional)
row_labels = ["Búsqueda en Anchura", "Búsqueda en Profundidad", "Búsqueda en Profundidad opt.", "Búsqueda Greedy", "Búsqueda A*"]
table_data_with_labels = [[label] + row for label, row in zip(row_labels, table_data)]

# Print table with row labels
print(tabulate(table_data_with_labels, headers=["Run"] + headers, tablefmt="grid"))

+------------------------------+------------------+------------------+------------------+---------------------+-------------+--------------+
| Run                          | solution_found   |   nodes_explored |   states_visited |   nodes_in_frontier |   max_depth |   cost_total |
+==============================+==================+==================+==================+=====================+=============+==============+
| Búsqueda en Anchura          | True             |             1351 |              233 |                 285 |          31 |           31 |
+------------------------------+------------------+------------------+------------------+---------------------+-------------+--------------+
| Búsqueda en Profundidad      | True             |              122 |              122 |                  63 |         121 |          121 |
+------------------------------+------------------+------------------+------------------+---------------------+-------------+--------------+
| Búsqueda en

## Impresión de soluciones y caminos

In [15]:
for nodos in solution_anchura.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 1 | 4 | 3>
<Node HanoiState: 5 2 1 | 4 3 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState: 1 | 4 3 2 | 5>
<Node HanoiState: 1 | 4 3 | 5 2>
<Node HanoiState:  | 4 3 | 5 2 1>
<Node HanoiState: 3 | 4 | 5 2 1>
<Node HanoiState: 3 | 4 1 | 5 2>
<Node HanoiState: 3 2 | 4 1 | 5>
<Node HanoiState: 3 2 1 | 4 | 5>
<Node HanoiState: 3 2 1 |  | 5 4>
<Node HanoiState: 3 2 |  | 5 4 1>
<Node HanoiState: 3 | 2 | 5 4 1>
<Node HanoiState: 3 | 2 1 | 5 4>
<Node HanoiState:  | 2 1 | 5 4 3>
<Node HanoiState: 1 | 2 | 5 4 

In [16]:
for act in solution_anchura.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3


In [17]:
for nodos in solution_prof.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 | 1 | 2>
<Node HanoiState: 5 4 3 |  | 2 1>
<Node HanoiState: 5 4 3 1 |  | 2>
<Node HanoiState: 5 4 3 1 | 2 | >
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 | 2 | 3 1>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 4 | 1 | 3 2>
<Node HanoiState: 5 4 2 | 1 | 3>
<Node HanoiState: 5 4 2 |  | 3 1>
<Node HanoiState: 5 4 2 1 |  | 3>
<Node HanoiState: 5 4 2 1 | 3 | >
<Node HanoiState: 5 4 2 | 3 | 1>
<Node HanoiState: 5 4 2 | 3 1 | >
<Node HanoiState: 5 4 | 3 1 | 2>
<Node HanoiState: 5 4 | 3 | 2 1>
<Node HanoiState: 5 4 1 | 3 | 2>
<Node HanoiState: 5 4 1 | 3 2 | >
<Node HanoiState: 5 4 | 3 2 | 1>
<Node HanoiState: 5 4 | 3 2 1 | >
<Node HanoiState: 5 | 3 2 1 | 4>
<Node HanoiState: 5 | 3 2 | 4 1>
<Node HanoiState: 5 1 | 3 2

In [18]:
for act in solution_prof.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from 3 to 2
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from 3 to 2
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 4 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 3 from 2 to 3
Move disk 1 from 1 to 3
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 1 from 3 to 1
Move disk 2 from

In [19]:
for nodos in solution_prof_smart.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 | 2 | 3 1>
<Node HanoiState: 5 4 2 |  | 3 1>
<Node HanoiState: 5 4 2 | 1 | 3>
<Node HanoiState: 5 4 | 1 | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 | 4 | 3 1>
<Node HanoiState: 5 | 4 2 | 3 1>
<Node HanoiState: 5 | 4 2 1 | 3>
<Node HanoiState: 5 3 | 4 2 1 | >
<Node HanoiState: 5 3 | 4 2 | 1>
<Node HanoiState: 5 3 2 | 4 | 1>
<Node HanoiState: 5 3 2 | 4 1 | >
<Node HanoiState: 5 3 | 4 1 | 2>
<Node HanoiState: 5 3 | 4 | 2 1>
<Node HanoiState: 5 | 4 3 | 2 1>
<Node HanoiState: 5 | 4 3 1 | 2>
<Node HanoiState: 5 2 | 4 3 1 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState:  | 4 3 2 | 5 1

In [20]:
for act in solution_prof_smart.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from

In [21]:
for nodos in solution_greedy.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 1 | 4 | 3>
<Node HanoiState: 5 2 1 | 4 3 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState: 1 | 4 3 2 | 5>
<Node HanoiState: 1 | 4 3 | 5 2>
<Node HanoiState:  | 4 3 | 5 2 1>
<Node HanoiState: 3 | 4 | 5 2 1>
<Node HanoiState: 3 | 4 1 | 5 2>
<Node HanoiState: 3 2 | 4 1 | 5>
<Node HanoiState: 3 2 1 | 4 | 5>
<Node HanoiState: 3 2 1 |  | 5 4>
<Node HanoiState: 3 2 |  | 5 4 1>
<Node HanoiState: 3 | 2 | 5 4 1>
<Node HanoiState: 3 | 2 1 | 5 4>
<Node HanoiState:  | 2 1 | 5 4 3>
<Node HanoiState: 1 | 2 | 5 4 

In [22]:
for act in solution_greedy.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3


In [23]:
for nodos in solution_a_estrella.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 1 | 4 | 3>
<Node HanoiState: 5 2 1 | 4 3 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState: 1 | 4 3 2 | 5>
<Node HanoiState: 1 | 4 3 | 5 2>
<Node HanoiState:  | 4 3 | 5 2 1>
<Node HanoiState: 3 | 4 | 5 2 1>
<Node HanoiState: 3 | 4 1 | 5 2>
<Node HanoiState: 3 2 | 4 1 | 5>
<Node HanoiState: 3 2 1 | 4 | 5>
<Node HanoiState: 3 2 1 |  | 5 4>
<Node HanoiState: 3 2 |  | 5 4 1>
<Node HanoiState: 3 | 2 | 5 4 1>
<Node HanoiState: 3 | 2 1 | 5 4>
<Node HanoiState:  | 2 1 | 5 4 3>
<Node HanoiState: 1 | 2 | 5 4 

In [24]:
for act in solution_a_estrella.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3


## Evalución de tiempos de ejecución y memoria consumida

### Anchura

In [25]:
%load_ext memory_profiler

In [26]:
%%timeit 
solution_anchura, metrics_anchura = breadth_first_search(number_disks=5)

37.5 ms ± 170 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%%memit
solution_anchura, metrics_anchura = breadth_first_search(number_disks=5)

peak memory: 74.81 MiB, increment: 0.92 MiB


### Búsqueda en profundidad

In [28]:
%%timeit 
solution_prof, metrics_prof = breadth_first_search(number_disks=5)

38.6 ms ± 372 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%%memit
solution_prof, metrics_prof = breadth_first_search(number_disks=5)

peak memory: 74.83 MiB, increment: 0.00 MiB


### Se nota que al no contar con información que indique que pieza se movió previamente, el algoritmo registra movimientos innecesarios, alargando el árbol de solucion.
Se buscará a continuación encontrar la pieza movida en el paso anterior, y si es diferente a la que el árbol sugiere mover, se procede, sino se saltea el paso.

### Búsqueda en profundidad Optimizado

In [30]:
%%timeit 
solution_prof_smart, metrics_prof_smart = depth_first_search_smart(number_disks=5)

2.34 ms ± 99.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
%%memit
solution_prof_smart, metrics_prof_smart = depth_first_search_smart(number_disks=5)

peak memory: 74.86 MiB, increment: 0.00 MiB


### Búsqueda Greedy

In [32]:
%%timeit 
solution_greedy, metrics_greedy = depth_first_search_smart(number_disks=5)

2.29 ms ± 28.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%%memit
solution_greedy, metrics_greedy = depth_first_search_smart(number_disks=5)

peak memory: 74.86 MiB, increment: 0.00 MiB


### Búsqueda A*

In [34]:
%%timeit 
solution_a_estrella, metrics_a_estrella = depth_first_search_smart(number_disks=5)

2.25 ms ± 15.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
%%memit
solution_a_estrella, metrics_a_estrella = depth_first_search_smart(number_disks=5)

peak memory: 74.86 MiB, increment: 0.00 MiB


## SIMULADOR:
A continuación generamos los json para poder correr el simulador

In [36]:
#Breadth:
solution_anchura.generate_solution_for_simulator("./sequence_breadth.json")

In [37]:
#Depth:
solution_prof.generate_solution_for_simulator("./sequence_depth.json")

In [38]:
#Depth_Smart:
solution_prof_smart.generate_solution_for_simulator("./sequence_depth_optimized.json")

In [39]:
#Gredy:
solution_greedy.generate_solution_for_simulator("./sequence_greedy.json")

In [40]:
#A*:
solution_a_estrella.generate_solution_for_simulator("./sequence_a_star.json")